In [0]:
!pip install scikit-learn --q
!pip install pandas --q
dbutils.library.restartPython()

In [0]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [0]:
spark_df = spark.table('bronze.sensus')
pandas_df = spark_df.toPandas()
pandas_df.head()

In [0]:
country_to_iso_numeric = {
    'United-States': 840,
    'Cuba': 192,
    'Jamaica': 388,
    'India': 356,
    'Not-Informed': 0,
    'Mexico': 484,
    'South Korea': 410,
    'Puerto-Rico': 630,
    'England': 826,  # parte do UK
    'Canada': 124,
    'Germany': 276,
    'Iran': 364,
    'Philippines': 608,
    'Italy': 380,
    'Poland': 616,
    'Columbia': 170,  # corrigido para Colombia
    'Cambodia': 116,
    'Thailand': 764,
    'Ecuador': 218,
    'Laos': 418,
    'Taiwan': 158,
    'Haiti': 332,
    'Portugal': 620,
    'Dominican-Republic': 214,
    'El-Salvador': 222,
    'France': 250,
    'Honduras': 340,
    'Guatemala': 320,
    'China': 156,
    'Japan': 392,
    'Yugoslavia': 890,  # código histórico
    'Peru': 604,
    'Outlying-US(Guam-USVI-etc)': 581,
    'Scotland': 826,  # parte do UK
    'Trinadad&Tobago': 780,
    'Greece': 300,
    'Nicaragua': 558,
    'Vietnam': 704,
    'Hong': 344,  # Hong Kong
    'Ireland': 372,
    'Hungary': 348,
    'South': 840,  # como United States (por instrução)
    'Holand-Netherlands': 528  # Netherlands
}


In [0]:
def set_target(df:pd.DataFrame, target:str='income', positive:str='>50K') -> pd.DataFrame:
    df[target] = (df[target] == positive).astype(int)
    return df

def drop_columns(df:pd.DataFrame, columns:list) -> pd.DataFrame:
    return df.drop(columns, axis=1)

def set_unemployed(df: pd.DataFrame) -> pd.DataFrame:
    mask = df['workclass'].eq('Never-worked')
    df.loc[mask, 'occupation'] = 'Unemployed'
    return df

def set_not_informed(df: pd.DataFrame) -> pd.DataFrame:
    mask = ((df['workclass'].eq('?') & df['occupation'].eq('?')) |\
         (df['workclass'].isna() & df['occupation'].isna()) )
    df.loc[mask, ['workclass', 'occupation']] = 'Not-Informed'
    return df

def set_country_not_informed(df: pd.DataFrame)->pd.DataFrame:
    mask = (df['native-country'].isna() | df['native-country'].eq('?'))
    df.loc[mask, 'native-country'] = 'Not-Informed'
    return df

def set_south_korea(df:pd.DataFrame)->pd.DataFrame:
    mask = ((df['native-country'].eq('South'))&(df['race'].eq('Asian-Pac-Islander')))
    df.loc[mask, 'native-country'] = 'South Korea'
    return df

In [0]:
column_to_drop = ['fnlwgt', 'capital-gain', 'capital-loss']
test = set_unemployed(pandas_df)
test = drop_columns(test, column_to_drop)
test = set_not_informed(test)
test = set_country_not_informed(test)
test = set_south_korea(test)
test.head()

In [0]:
occupation_list = test['occupation'].unique().tolist()
workclass_list = test['workclass'].unique().tolist()
native_country_list = test['native-country'].unique().tolist()
sex_list = test['sex'].unique().tolist()
marital_status_list = test['marital-status'].unique().tolist()
race_list = test['race'].unique().tolist()
education_list = test['education'].unique().tolist()
relationship_list = test['relationship'].unique().tolist()

In [0]:
print(f'Occupation: {occupation_list}\n')
print(f'Workclass: {workclass_list}\n')
print(f'Native Country: {native_country_list}\n')
print(f'Sex: {sex_list}\n')
print(f'Marital Status: {marital_status_list}\n')
print(f'Race: {race_list}\n')
print(f'Education: {education_list}\n')
print(f'Relationship: {relationship_list}\n') 

In [0]:
test.info()

In [0]:
data = test[(test['native-country'] == 'South')&(test['race'] == 'Black')]
data.head()

In [0]:
data['race'].unique()